# Making a function to add fields to the service

In [1]:
import arcgis
from arcgis.gis import GIS, GroupManager
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
from copy import deepcopy
from arcgis import geometry
import re
from pprint import pprint
import pandas as pd

In [2]:
env_path = ".env"
with open(env_path) as f:
    env = {}
    for line in f:
        env_key, _val = line.split("=")
        env_value = _val.split("\n")[0]
        env[env_key] = env_value

In [3]:
aol_password = env['aol_key']
aol_username = env['aol_username']

In [4]:
#gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

In [3]:
gis = GIS(profile = "eowilson")

In [4]:
def create_strict_reg_exp(to_search):
    try:
        reg_exp = f"^{to_search}$"
        #logging.info(f"regular expression is: {reg_exp}")
    except:
        print("There was a problem with the string.")
    return reg_exp

In [5]:
def findItemGetID(csvName, gis):
    try:
        searched_item = gis.content.search(csvName, item_type = "Feature Layer")
        for i in searched_item:
            reg_exp = create_strict_reg_exp(csvName)
            if re.search(reg_exp, i.title)!= None:    
                #logging.info(f"{csvName} has the id: {i.id}")
                return i.id
    except:
        print("There was a problem finding the item")

In [6]:
def getFlayerFromID(item_id):
    item = gis.content.get(item_id)
    flayer = item.layers[0]
    return flayer

In [7]:
def getSDFfromFlayer(flayer):
    sdf = pd.DataFrame.spatial.from_layer(flayer)
    return sdf

In [8]:
def createFieldsToBeAdded(flayer, csv_table):
    flayer_fields = flayer.manager.properties.fields
    template_field = dict(deepcopy(flayer_fields[0]))
    sdf = getSDFfromFlayer(flayer)
    new_field_names = list(csv_table.columns.difference(sdf.columns))
    
    fields_to_be_added = []
    for new_field_name in new_field_names:
        current_field = deepcopy(template_field)
        dt = csv_table[new_field_name].dtypes
        
        if dt == 'O':
            #put the type to character
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeString'
            #current_field['length'] = 8000
        if dt == 'float64':
            #put the type to double
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeDouble'
            #current_field['length'] = 8000      

        current_field['name'] = new_field_name.lower()
        current_field['alias'] = new_field_name
        current_field['nullable'] = True
        current_field['editable'] = True
        fields_to_be_added.append(current_field)
    return fields_to_be_added

In [9]:
def createFeaturesForUpdate(flayer, csv_table, fields_to_be_added, id_field_in_csv, id_field_in_service):
    fset2 = flayer.query()
    features2 = fset2.features
    features_for_update = []
    for country_id in csv_table[id_field_in_csv]:
        try:
            # get the matching row from csv
            matching_row = csv_table.where(csv_table[id_field_in_csv] == country_id).dropna()

            #print(str(country_id) + " Adding additional attributes for: " + matching_row['iso3'].values[0])

            # get the feature to be updated
            assert  len([f for f in features2 if f.attributes[id_field_in_service] == country_id]),  "id not matched"
            original_feature = [f for f in features2 if f.attributes[id_field_in_service] == country_id][0]
            feature_to_be_updated = deepcopy(original_feature)

            # assign the updated values
            for field in fields_to_be_added:
                feature_to_be_updated.attributes[field['name']] = matching_row[field['name']].values[0]
                #add this to the list of features to be updated
                features_for_update.append(feature_to_be_updated)
            #print(str(country_id) + " Done additional attributes for: " + matching_row['countryname'].values[0])
    
        except:
            print(f"{country_id} not available in service")
    return features_for_update

In [10]:
item_id = findItemGetID("gadm_centroid", gis)

In [11]:
flayer = getFlayerFromID(item_id)

In [49]:
csv_table_all = pd.read_csv("/Users/gretacvega/Documents/GitHub/vizzWork/MOL/gid_continent.csv")

In [51]:
csv_table = csv_table_all[['GID_0', 'continent']]

#### It is better if all the field names are lower case because otherwise the alias will have caps and the name won't which will make it tricky for the final function.

In [52]:
fields_to_be_added = createFieldsToBeAdded(flayer, csv_table)

In [53]:
fields_to_be_added

[{'name': 'continent',
  'type': 'esriFieldTypeString',
  'alias': 'continent',
  'sqlType': 'sqlTypeOther',
  'nullable': True,
  'editable': True,
  'visible': True,
  'domain': None,
  'defaultValue': None}]

In [54]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}

In [55]:
features_for_update = createFeaturesForUpdate(flayer, csv_table, fields_to_be_added, id_field_in_csv = "GID_0", id_field_in_service = "GID_0")

IOT not available in service
UMI not available in service


In [56]:
flayer.edit_features(updates= features_for_update)

{'addResults': [],
 'updateResults': [{'objectId': 1,
   'uniqueId': 1,
   'globalId': 'fe9f6eb0-f4f8-4f29-875a-5cbb3219e4e5',
   'success': True},
  {'objectId': 2,
   'uniqueId': 2,
   'globalId': '193ba976-0e5a-4cf6-9b09-d00bf83f4557',
   'success': True},
  {'objectId': 3,
   'uniqueId': 3,
   'globalId': '174ce788-4f67-4ae0-922f-d2ddac87f8c3',
   'success': True},
  {'objectId': 4,
   'uniqueId': 4,
   'globalId': '9f5f24d8-8b21-49a8-8f55-90b47cf63e7b',
   'success': True},
  {'objectId': 5,
   'uniqueId': 5,
   'globalId': '2b45351b-a335-490e-914e-7748d4f41f66',
   'success': True},
  {'objectId': 6,
   'uniqueId': 6,
   'globalId': '238963fe-66f4-49b3-ac6b-5ac57aade444',
   'success': True},
  {'objectId': 7,
   'uniqueId': 7,
   'globalId': '1c57acd6-8014-4086-8fdf-0c2476600d9c',
   'success': True},
  {'objectId': 8,
   'uniqueId': 8,
   'globalId': '63d38719-32e2-40d0-8d5a-2d84dd7bc2d3',
   'success': True},
  {'objectId': 9,
   'uniqueId': 9,
   'globalId': '0453843e-4d99-4f

# Adding new fields of information to National Report Cards centroid service

In [1]:
import arcgis
from arcgis.gis import GIS, GroupManager
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
from copy import deepcopy
from arcgis import geometry
import re
from pprint import pprint
import pandas as pd

In [2]:
env_path = ".env"
with open(env_path) as f:
    env = {}
    for line in f:
        env_key, _val = line.split("=")
        env_value = _val.split("\n")[0]
        env[env_key] = env_value

In [3]:
aol_password = env['aol_key']
aol_username = env['aol_username']

In [4]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)

In [13]:
def create_strict_reg_exp(to_search):
    try:
        reg_exp = f"^{to_search}$"
        #logging.info(f"regular expression is: {reg_exp}")
    except:
        print("There was a problem with the string.")
    return reg_exp

In [14]:
def findItemGetID(csvName, gis):
    try:
        searched_item = gis.content.search(csvName, item_type = "Feature Layer")
        for i in searched_item:
            reg_exp = create_strict_reg_exp(csvName)
            if re.search(reg_exp, i.title)!= None:    
                #logging.info(f"{csvName} has the id: {i.id}")
                return i.id
    except:
        print("There was a problem finding the item")

## Get item that has to be updated

In [15]:
item_id = findItemGetID("gadm_centroid", gis)

In [16]:
item = gis.content.get(item_id)

In [17]:
flayer = item.layers[0]

In [18]:
sdf = pd.DataFrame.spatial.from_layer(flayer)

In [15]:
type(sdf)

pandas.core.frame.DataFrame

## Process the csv with total number of species so it can have matching field with the service

In [4]:
n_by_taxa = pd.read_csv("data/Terrestrial_vertebrate_groups_by_country_20200617.csv")

In [5]:
n_by_taxa_wide = n_by_taxa.pivot_table(index=['iso3', 'countryname'], columns='speciesgroup', values='nspecies')

In [6]:
n_by_taxa_wide['nspecies'] = n_by_taxa_wide.sum(axis = 1)

In [7]:
n_by_taxa_wide.index.name = 'iso3'
n_by_taxa_wide.reset_index(inplace=True)

In [8]:
n_by_taxa_wide = n_by_taxa_wide.fillna(0)

In [9]:
n_by_taxa_wide.head()

speciesgroup,iso3,countryname,amphibians,birds,mammals,reptiles,nspecies
0,ABW,Aruba,3.0,92.0,4.0,32.0,131.0
1,AFG,Afghanistan,9.0,371.0,139.0,145.0,664.0
2,AGO,Angola,128.0,917.0,299.0,336.0,1680.0
3,AIA,Anguilla,2.0,106.0,5.0,12.0,125.0
4,ALA,Åland,5.0,112.0,11.0,4.0,132.0


In [10]:
n_by_taxa_wide.shape

(255, 7)

## Prepare the service to have more fields with the total number of species

In [19]:
#Get the existing list of fields on the published feature layer
flayer_fields = flayer.manager.properties.fields

In [20]:
for field in flayer_fields:
    print(f"{field.name:13}|  {field.alias:13}|  {field.type:25}|  {field.sqlType}")

OBJECTID_1   |  OBJECTID_1   |  esriFieldTypeOID         |  sqlTypeOther
GID_0        |  GID_0        |  esriFieldTypeString      |  sqlTypeOther
NAME_0       |  NAME_0       |  esriFieldTypeString      |  sqlTypeOther
jpg_url      |  jpg_url      |  esriFieldTypeString      |  sqlTypeOther
OBJECTID     |  OBJECTID     |  esriFieldTypeInteger     |  sqlTypeOther
GID          |  GID          |  esriFieldTypeString      |  sqlTypeOther
Area         |  Area         |  esriFieldTypeInteger     |  sqlTypeOther
GNI_PPP      |  GNI_PPP      |  esriFieldTypeDouble      |  sqlTypeOther
Protected    |  Protected    |  esriFieldTypeInteger     |  sqlTypeOther
HM_0         |  HM_0         |  esriFieldTypeInteger     |  sqlTypeOther
HM_low       |  HM_low       |  esriFieldTypeInteger     |  sqlTypeOther
HM_moderate  |  HM_moderate  |  esriFieldTypeInteger     |  sqlTypeOther
HM_high      |  HM_high      |  esriFieldTypeInteger     |  sqlTypeOther
Population2016|  SUM          |  esriFieldTypeDoubl

In [34]:
# get a template field with the type of interest in this case esriFieldTypeInteger
template_field = dict(deepcopy(flayer_fields[6]))
template_field

{'name': 'Area',
 'type': 'esriFieldTypeInteger',
 'alias': 'Area',
 'sqlType': 'sqlTypeOther',
 'nullable': True,
 'editable': True,
 'visible': True,
 'domain': None,
 'defaultValue': None}

In [23]:
# get the list of new fields to add from the third spreadsheet, that are not in spread sheets 1,2
new_field_names = list(n_by_taxa_wide.columns.difference(sdf.columns))
new_field_names
# when doing this there are two fields iso3 and country that are kept and they are not needed as the join of the tables is done feature by feature

['countryname', 'iso3']

In [35]:
new_field_names = ['amphibians', 'birds', 'mammals', 'nspecies', 'reptiles']
new_field_names

In [37]:
fields_to_be_added = []
for new_field_name in new_field_names:
    current_field = deepcopy(template_field)
    if new_field_name.lower() == 'class':
        current_field['sqlType'] = 'sqlTypeVarchar'
        current_field['type'] = 'esriFieldTypeInteger'
        #current_field['length'] = 8000
    current_field['name'] = new_field_name.lower()
    current_field['alias'] = new_field_name
    fields_to_be_added.append(current_field)
    
len(fields_to_be_added)

5

In [38]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}

In [39]:
new_fields = flayer.manager.properties.fields
len(new_fields)

31

In [40]:
for field in new_fields:
    print(f"{field.name:10}|  {field.type}")

OBJECTID_1|  esriFieldTypeOID
GID_0     |  esriFieldTypeString
NAME_0    |  esriFieldTypeString
jpg_url   |  esriFieldTypeString
OBJECTID  |  esriFieldTypeInteger
GID       |  esriFieldTypeString
Area      |  esriFieldTypeInteger
GNI_PPP   |  esriFieldTypeDouble
Protected |  esriFieldTypeInteger
HM_0      |  esriFieldTypeInteger
HM_low    |  esriFieldTypeInteger
HM_moderate|  esriFieldTypeInteger
HM_high   |  esriFieldTypeInteger
Population2016|  esriFieldTypeDouble
max_amph  |  esriFieldTypeInteger
max_bird  |  esriFieldTypeInteger
max_mamm  |  esriFieldTypeInteger
max_rept  |  esriFieldTypeInteger
max_cact  |  esriFieldTypeInteger
max_coni  |  esriFieldTypeInteger
max_all   |  esriFieldTypeInteger
sentence  |  esriFieldTypeString
prop_protected|  esriFieldTypeDouble
gadm_prop_COUNT|  esriFieldTypeInteger
N_SPECIES |  esriFieldTypeInteger
SPI       |  esriFieldTypeDouble
amphibians|  esriFieldTypeInteger
birds     |  esriFieldTypeInteger
mammals   |  esriFieldTypeInteger
nspecies  |  

## Update the data in the new fields with the data from the csv of total number of species

In [41]:
fset2 = flayer.query()
features2 = fset2.features

In [44]:
features_for_update = []
for country_id in n_by_taxa_wide['iso3']:
    try:
        # get the matching row from csv
        matching_row = n_by_taxa_wide.where(n_by_taxa_wide.iso3 == country_id).dropna()

        print(str(country_id) + " Adding additional attributes for: " + matching_row['iso3'].values[0])

        # get the feature to be updated
        assert  len([f for f in features2 if f.attributes['GID_0'] == country_id]),  "country not matched"
        original_feature = [f for f in features2 if f.attributes['GID_0'] == country_id][0]
        feature_to_be_updated = deepcopy(original_feature)

        # assign the updated values
        feature_to_be_updated.attributes['amphibians'] = matching_row['amphibians'].values[0]
        feature_to_be_updated.attributes['birds'] = int(matching_row['birds'])
        #feature_to_be_updated.attributes['countryname'] = str(matching_row['countryname'])
        feature_to_be_updated.attributes['mammals'] = int(matching_row['mammals'])
        #feature_to_be_updated.attributes['iso3'] = str(matching_row['iso3'])
        feature_to_be_updated.attributes['nspecies'] = int(matching_row['nspecies'])
        feature_to_be_updated.attributes['reptiles'] = int(matching_row['reptiles'])

        #add this to the list of features to be updated
        features_for_update.append(feature_to_be_updated)
        #print(str(country_id) + " Done additional attributes for: " + matching_row['countryname'].values[0])
    except:
        print(f"{country_id} not available in service")

ABW Adding additional attributes for: ABW
AFG Adding additional attributes for: AFG
AGO Adding additional attributes for: AGO
AIA Adding additional attributes for: AIA
ALA Adding additional attributes for: ALA
ALB Adding additional attributes for: ALB
AND Adding additional attributes for: AND
ARE Adding additional attributes for: ARE
ARG Adding additional attributes for: ARG
ARM Adding additional attributes for: ARM
ASM Adding additional attributes for: ASM
ATA Adding additional attributes for: ATA
ATF Adding additional attributes for: ATF
ATG Adding additional attributes for: ATG
AUS Adding additional attributes for: AUS
AUT Adding additional attributes for: AUT
AZE Adding additional attributes for: AZE
BDI Adding additional attributes for: BDI
BEL Adding additional attributes for: BEL
BEN Adding additional attributes for: BEN
BES Adding additional attributes for: BES
BFA Adding additional attributes for: BFA
BGD Adding additional attributes for: BGD
BGR Adding additional attributes f

In [45]:
flayer.edit_features(updates= features_for_update)

{'addResults': [],
 'updateResults': [{'objectId': 1,
   'uniqueId': 1,
   'globalId': None,
   'success': True},
  {'objectId': 2, 'uniqueId': 2, 'globalId': None, 'success': True},
  {'objectId': 3, 'uniqueId': 3, 'globalId': None, 'success': True},
  {'objectId': 4, 'uniqueId': 4, 'globalId': None, 'success': True},
  {'objectId': 5, 'uniqueId': 5, 'globalId': None, 'success': True},
  {'objectId': 6, 'uniqueId': 6, 'globalId': None, 'success': True},
  {'objectId': 7, 'uniqueId': 7, 'globalId': None, 'success': True},
  {'objectId': 8, 'uniqueId': 8, 'globalId': None, 'success': True},
  {'objectId': 9, 'uniqueId': 9, 'globalId': None, 'success': True},
  {'objectId': 10, 'uniqueId': 10, 'globalId': None, 'success': True},
  {'objectId': 11, 'uniqueId': 11, 'globalId': None, 'success': True},
  {'objectId': 12, 'uniqueId': 12, 'globalId': None, 'success': True},
  {'objectId': 13, 'uniqueId': 13, 'globalId': None, 'success': True},
  {'objectId': 14, 'uniqueId': 14, 'globalId': Non

# Preparing csv with number of endemic species

In [11]:
end_by_taxa = pd.read_csv("/Users/gretacvega/Documents/GitHub/he-scratchfolder/data/Terrestrial_vertebrate_totals_by_country_20200617_and_endemic_groups.csv")

In [12]:
end_by_taxa.head()

,countryname,iso3,nspecies,total_endemic,endemic_amphibians,endemic_birds,endemic_mammals,endemic_reptiles
0,Afghanistan,AFG,664,5,1,0,0,4
1,Akrotiri and Dhekelia,XAD,163,0,0,0,0,0
2,Åland,ALA,132,0,0,0,0,0
3,Albania,ALB,398,0,0,0,0,0
4,Algeria,DZA,516,4,1,1,1,1


In [13]:
end_by_taxa.shape

(255, 8)

In [14]:
flayer_fields = flayer.manager.properties.fields

In [15]:
for field in flayer_fields:
    print(f"{field.name:13}|  {field.alias:13}|  {field.type:25}|  {field.sqlType}")

OBJECTID_1   |  OBJECTID_1   |  esriFieldTypeOID         |  sqlTypeOther
GID_0        |  GID_0        |  esriFieldTypeString      |  sqlTypeOther
NAME_0       |  NAME_0       |  esriFieldTypeString      |  sqlTypeOther
jpg_url      |  jpg_url      |  esriFieldTypeString      |  sqlTypeOther
OBJECTID     |  OBJECTID     |  esriFieldTypeInteger     |  sqlTypeOther
GID          |  GID          |  esriFieldTypeString      |  sqlTypeOther
Area         |  Area         |  esriFieldTypeInteger     |  sqlTypeOther
GNI_PPP      |  GNI_PPP      |  esriFieldTypeDouble      |  sqlTypeOther
Protected    |  Protected    |  esriFieldTypeInteger     |  sqlTypeOther
HM_0         |  HM_0         |  esriFieldTypeInteger     |  sqlTypeOther
HM_low       |  HM_low       |  esriFieldTypeInteger     |  sqlTypeOther
HM_moderate  |  HM_moderate  |  esriFieldTypeInteger     |  sqlTypeOther
HM_high      |  HM_high      |  esriFieldTypeInteger     |  sqlTypeOther
Population2016|  SUM          |  esriFieldTypeDoubl

In [16]:
template_field = dict(deepcopy(flayer_fields[6]))
template_field

{'name': 'Area',
 'type': 'esriFieldTypeInteger',
 'alias': 'Area',
 'sqlType': 'sqlTypeOther',
 'nullable': True,
 'editable': True,
 'visible': True,
 'domain': None,
 'defaultValue': None}

In [17]:
new_field_names = ['total_endemic',	'endemic_amphibians',	'endemic_birds',	'endemic_mammals',	'endemic_reptiles']
new_field_names

['total_endemic',
 'endemic_amphibians',
 'endemic_birds',
 'endemic_mammals',
 'endemic_reptiles']

In [18]:
fields_to_be_added = []
for new_field_name in new_field_names:
    current_field = deepcopy(template_field)
    if new_field_name.lower() == 'class':
        current_field['sqlType'] = 'sqlTypeVarchar'
        current_field['type'] = 'esriFieldTypeInteger'
        #current_field['length'] = 8000
    current_field['name'] = new_field_name.lower()
    current_field['alias'] = new_field_name
    fields_to_be_added.append(current_field)
    
len(fields_to_be_added)

5

In [19]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}

In [20]:
new_fields = flayer.manager.properties.fields
len(new_fields)

36

In [21]:
for field in new_fields:
    print(f"{field.name:10}|  {field.type}")

OBJECTID_1|  esriFieldTypeOID
GID_0     |  esriFieldTypeString
NAME_0    |  esriFieldTypeString
jpg_url   |  esriFieldTypeString
OBJECTID  |  esriFieldTypeInteger
GID       |  esriFieldTypeString
Area      |  esriFieldTypeInteger
GNI_PPP   |  esriFieldTypeDouble
Protected |  esriFieldTypeInteger
HM_0      |  esriFieldTypeInteger
HM_low    |  esriFieldTypeInteger
HM_moderate|  esriFieldTypeInteger
HM_high   |  esriFieldTypeInteger
Population2016|  esriFieldTypeDouble
max_amph  |  esriFieldTypeInteger
max_bird  |  esriFieldTypeInteger
max_mamm  |  esriFieldTypeInteger
max_rept  |  esriFieldTypeInteger
max_cact  |  esriFieldTypeInteger
max_coni  |  esriFieldTypeInteger
max_all   |  esriFieldTypeInteger
sentence  |  esriFieldTypeString
prop_protected|  esriFieldTypeDouble
gadm_prop_COUNT|  esriFieldTypeInteger
N_SPECIES |  esriFieldTypeInteger
SPI       |  esriFieldTypeDouble
amphibians|  esriFieldTypeInteger
birds     |  esriFieldTypeInteger
mammals   |  esriFieldTypeInteger
nspecies  |  

# Update the data in the new fields with the data from the csv of total number of endemic species

In [22]:
fset2 = flayer.query()
features2 = fset2.features

In [25]:
features_for_update = []
for country_id in end_by_taxa['iso3']:
    try:
        # get the matching row from csv
        matching_row = end_by_taxa.where(end_by_taxa.iso3 == country_id).dropna()

        print(str(country_id) + " Adding additional attributes for: " + matching_row['iso3'].values[0])

        # get the feature to be updated
        assert  len([f for f in features2 if f.attributes['GID_0'] == country_id]),  "country not matched"
        original_feature = [f for f in features2 if f.attributes['GID_0'] == country_id][0]
        feature_to_be_updated = deepcopy(original_feature)

        # assign the updated values
        feature_to_be_updated.attributes['total_endemic'] = int(matching_row['total_endemic'])
        feature_to_be_updated.attributes['endemic_amphibians'] = int(matching_row['endemic_amphibians'])
        #feature_to_be_updated.attributes['countryname'] = str(matching_row['countryname'])
        feature_to_be_updated.attributes['endemic_birds'] = int(matching_row['endemic_birds'])
        #feature_to_be_updated.attributes['iso3'] = str(matching_row['iso3'])
        feature_to_be_updated.attributes['endemic_mammals'] = int(matching_row['endemic_mammals'])
        feature_to_be_updated.attributes['endemic_reptiles'] = int(matching_row['endemic_reptiles'])

        #add this to the list of features to be updated
        features_for_update.append(feature_to_be_updated)
        #print(str(country_id) + " Done additional attributes for: " + matching_row['countryname'].values[0])
    except:
        print(f"{country_id} not available in service")

AFG Adding additional attributes for: AFG
XAD Adding additional attributes for: XAD
ALA Adding additional attributes for: ALA
ALB Adding additional attributes for: ALB
DZA Adding additional attributes for: DZA
ASM Adding additional attributes for: ASM
AND Adding additional attributes for: AND
AGO Adding additional attributes for: AGO
AIA Adding additional attributes for: AIA
ATA Adding additional attributes for: ATA
ATG Adding additional attributes for: ATG
ARG Adding additional attributes for: ARG
ARM Adding additional attributes for: ARM
ABW Adding additional attributes for: ABW
AUS Adding additional attributes for: AUS
AUT Adding additional attributes for: AUT
AZE Adding additional attributes for: AZE
BHS Adding additional attributes for: BHS
BHR Adding additional attributes for: BHR
BGD Adding additional attributes for: BGD
BRB Adding additional attributes for: BRB
BLR Adding additional attributes for: BLR
BEL Adding additional attributes for: BEL
BLZ Adding additional attributes f

In [26]:
flayer.edit_features(updates= features_for_update)

{'addResults': [],
 'updateResults': [{'objectId': 2,
   'uniqueId': 2,
   'globalId': None,
   'success': True},
  {'objectId': 241, 'uniqueId': 241, 'globalId': None, 'success': True},
  {'objectId': 5, 'uniqueId': 5, 'globalId': None, 'success': True},
  {'objectId': 6, 'uniqueId': 6, 'globalId': None, 'success': True},
  {'objectId': 65, 'uniqueId': 65, 'globalId': None, 'success': True},
  {'objectId': 11, 'uniqueId': 11, 'globalId': None, 'success': True},
  {'objectId': 7, 'uniqueId': 7, 'globalId': None, 'success': True},
  {'objectId': 3, 'uniqueId': 3, 'globalId': None, 'success': True},
  {'objectId': 4, 'uniqueId': 4, 'globalId': None, 'success': True},
  {'objectId': 12, 'uniqueId': 12, 'globalId': None, 'success': True},
  {'objectId': 14, 'uniqueId': 14, 'globalId': None, 'success': True},
  {'objectId': 9, 'uniqueId': 9, 'globalId': None, 'success': True},
  {'objectId': 10, 'uniqueId': 10, 'globalId': None, 'success': True},
  {'objectId': 1, 'uniqueId': 1, 'globalId':

# adding data from the spi csv (adding protection needed)

In [11]:
spi_df = pd.read_csv("/Users/gretacvega/Documents/GitHub/he-scratchfolder/data/National_report_cards_terrestrial_20200330.csv")

In [12]:
spi_df.head()

,GID_0,NAME_0,AREA_KM2,PERC_PROT,PERC_NEED,N_SPECIES,SPI
0,ABW,Aruba,181.94,0.00,78.07,131,21.47
1,AFG,Afghanistan,643857.50,0.39,46.87,664,18.04
2,AGO,Angola,1247421.82,1.15,55.72,1680,66.63
3,AIA,Anguilla,83.30,0.16,84.47,125,0.10
4,ALA,Åland,1506.31,0.15,1.49,132,55.41


## Prepare the service to have more fields with the total number of species

In [13]:
#Get the existing list of fields on the published feature layer
flayer_fields = flayer.manager.properties.fields

In [14]:
for field in flayer_fields:
    print(f"{field.name:13}|  {field.alias:13}|  {field.type:25}|  {field.sqlType}")

OBJECTID_1   |  OBJECTID_1   |  esriFieldTypeOID         |  sqlTypeOther
GID_0        |  GID_0        |  esriFieldTypeString      |  sqlTypeOther
NAME_0       |  NAME_0       |  esriFieldTypeString      |  sqlTypeOther
jpg_url      |  jpg_url      |  esriFieldTypeString      |  sqlTypeOther
OBJECTID     |  OBJECTID     |  esriFieldTypeInteger     |  sqlTypeOther
GID          |  GID          |  esriFieldTypeString      |  sqlTypeOther
Area         |  Area         |  esriFieldTypeInteger     |  sqlTypeOther
GNI_PPP      |  GNI_PPP      |  esriFieldTypeDouble      |  sqlTypeOther
Protected    |  Protected    |  esriFieldTypeInteger     |  sqlTypeOther
HM_0         |  HM_0         |  esriFieldTypeInteger     |  sqlTypeOther
HM_low       |  HM_low       |  esriFieldTypeInteger     |  sqlTypeOther
HM_moderate  |  HM_moderate  |  esriFieldTypeInteger     |  sqlTypeOther
HM_high      |  HM_high      |  esriFieldTypeInteger     |  sqlTypeOther
Population2016|  SUM          |  esriFieldTypeDoubl

In [15]:
template_field = dict(deepcopy(flayer_fields[7]))
template_field

{'name': 'GNI_PPP',
 'type': 'esriFieldTypeDouble',
 'alias': 'GNI_PPP',
 'sqlType': 'sqlTypeOther',
 'nullable': True,
 'editable': True,
 'visible': True,
 'domain': None,
 'defaultValue': None}

In [16]:
new_field_names = ['protection_needed']

In [17]:
fields_to_be_added = []
for new_field_name in new_field_names:
    current_field = deepcopy(template_field)
    if new_field_name.lower() == 'class':
        current_field['sqlType'] = 'sqlTypeVarchar'
        current_field['type'] = 'esriFieldTypeInteger'
        #current_field['length'] = 8000
    current_field['name'] = new_field_name.lower()
    current_field['alias'] = new_field_name
    fields_to_be_added.append(current_field)
    
len(fields_to_be_added)

1

In [19]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}

In [20]:
new_fields = flayer.manager.properties.fields
len(new_fields)

37

In [21]:
for field in new_fields:
    print(f"{field.name:10}|  {field.type}")

OBJECTID_1|  esriFieldTypeOID
GID_0     |  esriFieldTypeString
NAME_0    |  esriFieldTypeString
jpg_url   |  esriFieldTypeString
OBJECTID  |  esriFieldTypeInteger
GID       |  esriFieldTypeString
Area      |  esriFieldTypeInteger
GNI_PPP   |  esriFieldTypeDouble
Protected |  esriFieldTypeInteger
HM_0      |  esriFieldTypeInteger
HM_low    |  esriFieldTypeInteger
HM_moderate|  esriFieldTypeInteger
HM_high   |  esriFieldTypeInteger
Population2016|  esriFieldTypeDouble
max_amph  |  esriFieldTypeInteger
max_bird  |  esriFieldTypeInteger
max_mamm  |  esriFieldTypeInteger
max_rept  |  esriFieldTypeInteger
max_cact  |  esriFieldTypeInteger
max_coni  |  esriFieldTypeInteger
max_all   |  esriFieldTypeInteger
sentence  |  esriFieldTypeString
prop_protected|  esriFieldTypeDouble
gadm_prop_COUNT|  esriFieldTypeInteger
N_SPECIES |  esriFieldTypeInteger
SPI       |  esriFieldTypeDouble
amphibians|  esriFieldTypeInteger
birds     |  esriFieldTypeInteger
mammals   |  esriFieldTypeInteger
nspecies  |  

## Update the data in the new fields with the data from the csv of total number of species

In [52]:
fset2 = flayer.query()
features2 = fset2.features

In [59]:
features_for_update = []
for country_id in spi_df['GID_0']:
    try:
        # get the matching row from csv
        matching_row = spi_df.where(spi_df.GID_0 == country_id).dropna()

        print(str(country_id) + " Adding additional attributes for: " + matching_row['GID_0'].values[0])

        # get the feature to be updated
        assert  len([f for f in features2 if f.attributes['GID_0'] == country_id]),  "country not matched"
        original_feature = [f for f in features2 if f.attributes['GID_0'] == country_id][0]
        feature_to_be_updated = deepcopy(original_feature)

        # assign the updated values
        #print( matching_row['PERC_NEED'].values[0])
        feature_to_be_updated.attributes['protection_needed'] = matching_row['PERC_NEED'].values[0]

        #add this to the list of features to be updated
        
        features_for_update.append(feature_to_be_updated)
        #print(str(country_id) + " Done additional attributes for: " + matching_row['countryname'].values[0])
    except:
        print(f"{country_id} not available in service")

ABW Adding additional attributes for: ABW
AFG Adding additional attributes for: AFG
AGO Adding additional attributes for: AGO
AIA Adding additional attributes for: AIA
ALA Adding additional attributes for: ALA
ALB Adding additional attributes for: ALB
AND Adding additional attributes for: AND
ARE Adding additional attributes for: ARE
ARG Adding additional attributes for: ARG
ARM Adding additional attributes for: ARM
ASM Adding additional attributes for: ASM
ATA Adding additional attributes for: ATA
ATF Adding additional attributes for: ATF
ATG Adding additional attributes for: ATG
AUS Adding additional attributes for: AUS
AUT Adding additional attributes for: AUT
AZE Adding additional attributes for: AZE
BDI Adding additional attributes for: BDI
BEL Adding additional attributes for: BEL
BEN Adding additional attributes for: BEN
BES Adding additional attributes for: BES
BFA Adding additional attributes for: BFA
BGD Adding additional attributes for: BGD
BGR Adding additional attributes f

In [60]:
flayer.edit_features(updates= features_for_update)

{'addResults': [],
 'updateResults': [{'objectId': 1,
   'uniqueId': 1,
   'globalId': None,
   'success': True},
  {'objectId': 2, 'uniqueId': 2, 'globalId': None, 'success': True},
  {'objectId': 3, 'uniqueId': 3, 'globalId': None, 'success': True},
  {'objectId': 4, 'uniqueId': 4, 'globalId': None, 'success': True},
  {'objectId': 5, 'uniqueId': 5, 'globalId': None, 'success': True},
  {'objectId': 6, 'uniqueId': 6, 'globalId': None, 'success': True},
  {'objectId': 7, 'uniqueId': 7, 'globalId': None, 'success': True},
  {'objectId': 8, 'uniqueId': 8, 'globalId': None, 'success': True},
  {'objectId': 9, 'uniqueId': 9, 'globalId': None, 'success': True},
  {'objectId': 10, 'uniqueId': 10, 'globalId': None, 'success': True},
  {'objectId': 11, 'uniqueId': 11, 'globalId': None, 'success': True},
  {'objectId': 12, 'uniqueId': 12, 'globalId': None, 'success': True},
  {'objectId': 13, 'uniqueId': 13, 'globalId': None, 'success': True},
  {'objectId': 14, 'uniqueId': 14, 'globalId': Non

# Making a function 

In [1]:
import arcgis
from arcgis.gis import GIS, GroupManager
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
from copy import deepcopy
from arcgis import geometry
import re
from pprint import pprint
import pandas as pd

In [2]:
env_path = ".env"
with open(env_path) as f:
    env = {}
    for line in f:
        env_key, _val = line.split("=")
        env_value = _val.split("\n")[0]
        env[env_key] = env_value

In [3]:
aol_password = env['aol_key']
aol_username = env['aol_username']

In [4]:
#gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

In [2]:
gis = GIS(profile = "eowilson")

In [5]:
def create_strict_reg_exp(to_search):
    try:
        reg_exp = f"^{to_search}$"
        #logging.info(f"regular expression is: {reg_exp}")
    except:
        print("There was a problem with the string.")
    return reg_exp

In [6]:
def findItemGetID(csvName, gis):
    try:
        searched_item = gis.content.search(csvName, item_type = "Feature Layer")
        for i in searched_item:
            reg_exp = create_strict_reg_exp(csvName)
            if re.search(reg_exp, i.title)!= None:    
                #logging.info(f"{csvName} has the id: {i.id}")
                return i.id
    except:
        print("There was a problem finding the item")

In [43]:
def getFlayerFromID(item_id):
    item = gis.content.get(item_id)
    flayer = item.layers[0]
    return flayer

In [44]:
def getSDFfromFlayer(flayer):
    sdf = pd.DataFrame.spatial.from_layer(flayer)
    return sdf

In [51]:
def createFieldsToBeAdded(flayer, csv_table):
    flayer_fields = flayer.manager.properties.fields
    template_field = dict(deepcopy(flayer_fields[0]))
    sdf = getSDFfromFlayer(flayer)
    new_field_names = list(csv_table.columns.difference(sdf.columns))
    
    fields_to_be_added = []
    for new_field_name in new_field_names:
        current_field = deepcopy(template_field)
        dt = csv_table[new_field_name].dtypes
        
        if dt == 'O':
            #put the type to character
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeString'
            #current_field['length'] = 8000
        if dt == 'float64':
            #put the type to double
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeDouble'
            #current_field['length'] = 8000      

        current_field['name'] = new_field_name.lower()
        current_field['alias'] = new_field_name
        current_field['nullable'] = True
        current_field['editable'] = True
        fields_to_be_added.append(current_field)
    return fields_to_be_added

In [61]:
def createFeaturesForUpdate(flayer, csv_table, fields_to_be_added, id_field_in_csv, id_field_in_service):
    fset2 = flayer.query()
    features2 = fset2.features
    features_for_update = []
    for country_id in csv_table[id_field_in_csv]:
        try:
            # get the matching row from csv
            matching_row = csv_table.where(csv_table[id_field_in_csv] == country_id).dropna()

            #print(str(country_id) + " Adding additional attributes for: " + matching_row['iso3'].values[0])

            # get the feature to be updated
            assert  len([f for f in features2 if f.attributes[id_field_in_service] == country_id]),  "id not matched"
            original_feature = [f for f in features2 if f.attributes[id_field_in_service] == country_id][0]
            feature_to_be_updated = deepcopy(original_feature)

            # assign the updated values
            for field in fields_to_be_added:
                feature_to_be_updated.attributes[field['name']] = matching_row[field['name']].values[0]
                #add this to the list of features to be updated
                features_for_update.append(feature_to_be_updated)
            #print(str(country_id) + " Done additional attributes for: " + matching_row['countryname'].values[0])
    
        except:
            print(f"{country_id} not available in service")
    return features_for_update

In [45]:
item_id = findItemGetID("gadm_centroid", gis)

In [46]:
flayer = getFlayerFromID(item_id)

In [47]:
csv_table = pd.read_csv("/Users/gretacvega/Desktop/svg_test.csv")

In [62]:
csv_table

,GID_0,svg
0,USA,"<svg xmlns=""http://www.w3.org/2000/svg"" xmlns:..."


In [52]:
fields_to_be_added = createFieldsToBeAdded(flayer, csv_table)

In [54]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}

In [63]:
features_for_update = createFeaturesForUpdate(flayer, csv_table, fields_to_be_added, id_field_in_csv = "GID_0", id_field_in_service = "GID_0")

In [65]:
flayer.edit_features(updates= features_for_update)


Field svg has invalid html content.


RuntimeError: 
Field svg has invalid html content.
(Error Code: 400)